In [1]:
import numpy as np
import sympy
from sympy import Symbol

In [2]:
class Interval:
    def __init__(self, lower_bound, upper_bound):
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound

    def __add__(self, other):
        if not isinstance(other, Interval):
            return Interval(lower_bound=self.lower_bound + other,
                            upper_bound=self.upper_bound + other)

        return Interval(lower_bound=self.lower_bound + other.lower_bound,
                        upper_bound=self.upper_bound + other.upper_bound)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        if not isinstance(other, Interval):
            return Interval(lower_bound=self.lower_bound - other,
                            upper_bound=self.upper_bound - other)
        return Interval(lower_bound=self.lower_bound - other.upper_bound,
                        upper_bound=self.upper_bound - other.lower_bound)

    def __rsub__(self, other):
        return -1 * self + other

    def __mul__(self, other):
        if not isinstance(other, Interval):
            return Interval(lower_bound=self.lower_bound * other,
                            upper_bound=self.upper_bound * other)

        term0 = self.lower_bound * other.lower_bound
        term1 = self.lower_bound * other.upper_bound
        term2 = self.upper_bound * other.lower_bound
        term3 = self.upper_bound * other.upper_bound

        terms = [term0, term1, term2, term3]

        constraints = []
        intervals = []

        for lower_idx, lower in enumerate(terms):
            for upper_idx, upper in enumerate(terms):
                print(lower_idx, upper_idx)
                if lower_idx != upper_idx:
                    lower_constraint = lower <= lower
                    upper_constraint = upper >= upper
                    for lower_tmp_idx, lower_tmp in enumerate(terms):
                        if lower_tmp_idx != lower_idx:
                            lower_constraint = lower_constraint & (lower <= lower_tmp)
                    for upper_tmp_idx, upper_tmp in enumerate(terms):
                        if upper_tmp_idx != upper_idx:
                            upper_constraint = upper_constraint & (upper >= upper_tmp)
                    print(lower_constraint, upper_constraint)
                    constraints.append(lower_constraint & upper_constraint)
                    intervals.append(Interval(lower, upper))

        return ConstrainedIntervals(constraints, intervals)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __truediv__(self, other):
        if not isinstance(other, Interval):
            return Interval(lower_bound=self.lower_bound / other,
                            upper_bound=self.upper_bound / other)

        constraints = []
        intervals = []

        constraints.append(((other.lower_bound > 0) & (other.upper_bound > 0)) | ((other.lower_bound < 0) & (other.upper_bound < 0)))
        intervals.append(Interval(1.0/other.upper_bound, 1.0/other.lower_bound))
        constraints.append((other.upper_bound == 0) & (other.lower_bound != 0))
        intervals.append(Interval(-np.inf, 1.0/other.lower_bound))
        constraints.append((other.lower_bound == 0) & (other.upper_bound != 0))
        intervals.append(Interval(1.0/other.upper_bound, np.inf))
        constraints.append((other.lower_bound < 0) & (other.upper_bound > 0))
        intervals.append(Interval(-np.inf, 1.0/other.lower_bound))
        constraints.append((other.lower_bound < 0) & (other.upper_bound > 0))
        intervals.append(Interval(1.0/other.upper_bound, np.inf))

        inverse = ConstrainedIntervals(constraints, intervals)

        return inverse * self

    def __rtruediv__(self, other):
        if not isinstance(other, (Interval,)):
            other = Interval(other, other)
        return other.__truediv__(self)

    def __repr__(self):
        return str(f"[{str(self.lower_bound)},\n{str(self.upper_bound)}]")

class ConstrainedIntervals:
    def __init__(self, constraints=None, intervals=None):
        if constraints is None:
            constraints = []
        elif not isinstance(constraints, list):
            constraints = [constraints]

        self.constraints = constraints

        if intervals is None:
            intervals = []
        elif not isinstance(intervals, list):
            intervals = [intervals]

        self.intervals = intervals

    def __add__(self, other):
        if not isinstance(other, (Interval, ConstrainedIntervals)):
            other = Interval(other, other)
        if isinstance(other, Interval):
            other = ConstrainedIntervals([True], [other])

        final_constraints = []
        final_intervals = []

        for const_0, int_0 in zip(self.constraints, self.intervals, strict=False):
            for const_1, int_1 in zip(other.constraints, other.intervals, strict=False):
                final_constraints.append(const_0 & const_1)
                final_intervals.append(int_0 + int_1)

        return ConstrainedIntervals(final_constraints, final_intervals)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        if not isinstance(other, (Interval, ConstrainedIntervals)):
            other = Interval(other, other)
        if isinstance(other, Interval):
            other = ConstrainedIntervals([True], [other])

        final_constraints = []
        final_intervals = []

        for const_0, int_0 in zip(self.constraints, self.intervals, strict=False):
            for const_1, int_1 in zip(other.constraints, other.intervals, strict=False):
                final_constraints.append(const_0 & const_1)
                final_intervals.append(int_0 - int_1)

        return ConstrainedIntervals(final_constraints, final_intervals)

    def __rsub__(self, other):
        return -1 * self + other

    def __mul__(self, other):
        if not isinstance(other, (Interval, ConstrainedIntervals)):
            other = Interval(other, other)
        if isinstance(other, Interval):
            other = ConstrainedIntervals([True], [other])

        final_constraints = []
        final_intervals = []

        for const_0, int_0 in zip(self.constraints, self.intervals, strict=False):
            for const_1, int_1 in zip(other.constraints, other.intervals, strict=False):
                res = int_0 * int_1
                for constraint, interval in zip(res.constraints, res.intervals, strict=False):
                    final_constraints.append(const_0 & const_1 & constraint)
                    final_intervals.append(interval)

        return ConstrainedIntervals(final_constraints, final_intervals)

    def __rmul__(self, other):
        print('rmul ci')
        return other.__mul__(self)

    def __truediv__(self, other):
        if not isinstance(other, (Interval, ConstrainedIntervals)):
            other = Interval(other, other)
        if isinstance(other, Interval):
            other = ConstrainedIntervals([True], [other])

        final_constraints = []
        final_intervals = []

        for const_0, int_0 in zip(self.constraints, self.intervals, strict=False):
            for const_1, int_1 in zip(other.constraints, other.intervals, strict=False):
                res = int_0 / int_1

                for constraint, interval in zip(res.constraints, res.intervals, strict=False):
                    final_constraints.append(const_0 & const_1 & constraint)
                    final_intervals.append(interval)

        return ConstrainedIntervals(final_constraints, final_intervals)

    def __rtruediv__(self, other):
        if not isinstance(other, (Interval, ConstrainedIntervals)):
            other = Interval(other, other)
        if isinstance(other, Interval):
            other = ConstrainedIntervals([True], [other])
        return other / self

    def __repr__(self):
        return str(f"[{str(self.constraints)},\n{str(self.intervals)}]")


In [3]:
tp = Symbol('tp')
tn = Symbol('tn')
fp = Symbol('fp')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
sens = Symbol('sens')
spec = Symbol('spec')
acc = Symbol('acc')
ppv = Symbol('ppv')
npv = Symbol('npv')

In [4]:
sens_e = tp/p
spec_e = tn/n
acc_e = (tp + tn)/(p + n)
ppv_e = tp/(tp + fp)
npv_e = tn/(tn + fn)

In [5]:
solutions = sympy.solve([ppv_e - ppv, npv_e - npv, p-tp-fn, n-tn-fp], [tp, tn, fp, fn])

In [6]:
solutions

{tp: ppv*(n*npv - n + npv*p)/(npv + ppv - 1),
 tn: npv*(n*ppv + p*ppv - p)/(npv + ppv - 1),
 fp: (-n*npv*ppv + n*npv + n*ppv - n - npv*p*ppv + npv*p)/(npv + ppv - 1),
 fn: (-n*npv*ppv + n*ppv - npv*p*ppv + npv*p + p*ppv - p)/(npv + ppv - 1)}

In [11]:
str(solutions[tp])

'ppv*(n*npv - n + npv*p)/(npv + ppv - 1)'

In [7]:
solutions[tp]

ppv*(n*npv - n + npv*p)/(npv + ppv - 1)

In [8]:
isens = Interval(sens-eps, sens+eps)
ispec = Interval(spec-eps, spec+eps)
ippv = Interval(ppv-eps, ppv+eps)
inpv = Interval(npv-eps, npv+eps)

In [11]:
tp = ippv*(n*inpv - n + inpv*p)/(inpv + ippv - 1)

rmul
mul
sub
mul
add
mul
0 0
0 1
((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)))
0 2
((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + 

In [14]:
p_r=10
n_r=20
tp_r=5
fn_r=p_r - tp_r
tn_r=15
fp_r=n_r - tn_r

npv_r = np.round(tn_r/(tn_r + fn_r), 4)
ppv_r = np.round(tp_r/(tp_r + fp_r), 4)
eps_r=0.0001

In [15]:
results = [constraint.subs({'npv': npv_r, 'ppv': ppv_r, 'eps': eps_r, 'p': p_r, 'n': n_r}) for constraint in tp.constraints]

In [23]:
np.sum(np.array(results) == True)

1

In [24]:
tp_int = tp.intervals[np.where(np.array(results) == True)[0][0]]

In [27]:
tp_int.lower_bound.subs({'npv': npv_r, 'ppv': ppv_r, 'eps': eps_r, 'p': p_r, 'n': n_r})

4.98900999200640

In [28]:
tp_int.upper_bound.subs({'npv': npv_r, 'ppv': ppv_r, 'eps': eps_r, 'p': p_r, 'n': n_r})

5.01101000800640

In [12]:
len(tp.constraints)

720

In [12]:
tmp = 1.0/inpv

rtruediv
truediv
mul ci
[((eps + npv > 0) & (-eps + npv > 0)) | ((eps + npv < 0) & (-eps + npv < 0)), False, False, (eps + npv > 0) & (-eps + npv < 0), (eps + npv > 0) & (-eps + npv < 0)]
[True]
mul
0 0
0 1
(1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) (1.0/(eps + npv) >= 1.0/(eps + npv)) & (1.0/(eps + npv) >= 1.0/(-eps + npv))
0 2
(1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) (1.0/(-eps + npv) >= 1.0/(eps + npv)) & (1.0/(-eps + npv) >= 1.0/(-eps + npv))
0 3
(1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) (1.0/(-eps + npv) >= 1.0/(eps + npv)) & (1.0/(-eps + npv) >= 1.0/(-eps + npv))
1 0
(1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) (1.0/(eps + npv) >= 1.0/(eps + npv)) & (1.0/(eps + npv) >= 1.0/(-eps + npv))
1 1
1 2
(1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) (1.0/(-eps + npv) >= 1.0/(eps + npv)) & (1.0/(-eps + npv) >= 1.0/(-eps + npv

In [13]:
tmp.__class__

__main__.ConstrainedIntervals

In [14]:
tmp.constraints[0]

(1.0/(eps + npv) >= 1.0/(eps + npv)) & (1.0/(eps + npv) <= 1.0/(eps + npv)) & (1.0/(eps + npv) >= 1.0/(-eps + npv)) & (1.0/(eps + npv) <= 1.0/(-eps + npv)) & (((eps + npv > 0) & (-eps + npv > 0)) | ((eps + npv < 0) & (-eps + npv < 0)))

In [15]:
tmp.intervals

[[1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [-inf,
 -inf],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 -inf],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 inf],
 [inf,
 1.0/(eps

In [16]:
tmp.intervals

[[1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(eps + npv)],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [-inf,
 -inf],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 -inf],
 [-inf,
 1.0/(-eps + npv)],
 [-inf,
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 -inf],
 [1.0/(-eps + npv),
 1.0/(-eps + npv)],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 1.0/(eps + npv)],
 [1.0/(eps + npv),
 inf],
 [1.0/(eps + npv),
 inf],
 [inf,
 1.0/(eps

In [17]:
ippv*(n*inpv - n + inpv*p)/(inpv + ippv - 1)

rmul
mul
sub
mul
add
mul
0 0
0 1
((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) & ((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv))) & ((-eps + ppv)*(n*(eps + npv) - n + p*(eps + npv)) >= (-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)))
0 2
((-eps + ppv)*(n*(-eps + npv) - n + p*(-eps + npv)) <= (eps + ppv)*(n*(eps + npv) - n + p*(eps + npv))) & ((-eps + 

In [ ]:
solutions[tn]

n*spec

In [ ]:
p * Interval(sens - eps, sens + eps)

[-eps + p + sens,
eps + p + sens]
subject to []

In [ ]:
solutions[tp].subs(
    {sens: Interval(sens-eps, sens+eps)})

p*sens

In [ ]:
2 + int0

[-eps + tp + 2,
eps + tp + 2]
subject to []

In [ ]:
int0*int1

[(-eps + tn)*(-eps + tp),
(eps + tn)*(eps + tp)]
subject to ['-eps + tn >= 0', '-eps + tp >= 0']

In [ ]:
int0 + int1 + p

[-2*eps + p + tn + tp,
2*eps + p + tn + tp]
subject to []

In [ ]:
int0/int1

TypeError: cannot determine truth value of Relational

In [ ]:
sens = Symbol('sens')
spec = Symbol('spec')
ppv = Symbol('ppv')
npv = Symbol('npv')
acc = Symbol('acc')
tp = Symbol('tp')
tn = Symbol('tn')
fp = Symbol('fp')
fn = Symbol('fn')
p = Symbol('p')
n = Symbol('n')

In [ ]:
results = sympy.solve([ppv - tp/(tp + fp), npv - (tn)/(tn + fn), p - tp - fn, n - tn - fp], [tp, tn, fp, fn])

In [ ]:
results

{tp: ppv*(n*npv - n + npv*p)/(npv + ppv - 1),
 tn: npv*(n*ppv + p*ppv - p)/(npv + ppv - 1),
 fp: (-n*npv*ppv + n*npv + n*ppv - n - npv*p*ppv + npv*p)/(npv + ppv - 1),
 fn: (-n*npv*ppv + n*ppv - npv*p*ppv + npv*p + p*ppv - p)/(npv + ppv - 1)}

In [ ]:
sympy.factor(results[fn]).subs({tp: results[tp]})

-(npv - 1)*(n*ppv + p*ppv - p)/(npv + ppv - 1)